# Practical example. Audiobooks

## Create the machine learning algorithm



### Import the relevant libraries

In [1]:
# we must import the libraries once again since we haven't imported them in this file
import numpy as np
import tensorflow as tf

### Data

In [2]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('../data/Audiobooks_data_train.npz')

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype("float")
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype("int")

# we load the validation data in the temporary variable
npz = np.load('../data/Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype("float"), npz['targets'].astype("int")

# we load the test data in the temporary variable
npz = np.load('../data/Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype("float"), npz['targets'].astype("int")

### Model
Outline, optimizers, loss, early stopping and training

In [3]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

csv_logger = tf.keras.callbacks.CSVLogger("../run/log.csv", separator=',', append=False)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping, csv_logger], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Epoch 1/100
36/36 - 2s - 66ms/step - accuracy: 0.7673 - loss: 0.5584 - val_accuracy: 0.8814 - val_loss: 0.4321
Epoch 2/100
36/36 - 0s - 11ms/step - accuracy: 0.8751 - loss: 0.3705 - val_accuracy: 0.8837 - val_loss: 0.3304
Epoch 3/100
36/36 - 0s - 9ms/step - accuracy: 0.8818 - loss: 0.3175 - val_accuracy: 0.8881 - val_loss: 0.3084
Epoch 4/100
36/36 - 0s - 4ms/step - accuracy: 0.8874 - loss: 0.2972 - val_accuracy: 0.8926 - val_loss: 0.2935
Epoch 5/100
36/36 - 0s - 7ms/step - accuracy: 0.8924 - loss: 0.2851 - val_accuracy: 0.8971 - val_loss: 0.2858
Epoch 6/100
36/36 - 0s - 5ms/step - accuracy: 0.8952 - loss: 0.2754 - val_accuracy: 0.9016 - val_loss: 0.2752
Epoch 7/100
36/36 - 0s - 5ms/step - accuracy: 0.8983 - loss: 0.2676 - val_accuracy: 0.9038 - val_loss: 0.2731
Epoch 8/100
36/36 - 0s - 4ms/step - accuracy: 0.9003 - loss: 0.2622 - val_accuracy: 0.9038 - val_loss: 0.2737
Epoch 9/100
36/36 - 0s - 5ms/step - accuracy: 0.8997 - loss: 0.2587 - val_accuracy: 0.9083 - val_loss: 0.2656
Epoch 10

## Model Evaluation and Testing

After training your model on the training data and validating it using a separate validation set, the next crucial step is to test its final prediction performance using a test dataset—one that the model has never encountered before.

Why is this so important? The test set serves as the ultimate benchmark for your model’s ability to generalize. If you've fine-tuned your model and adjusted hyperparameters based on the validation data, using the test data for any adjustments can lead to overfitting, meaning your model will perform well only on the test set, not on new, unseen data.

The key takeaway is that testing should be the final step. Once you run your model on the test data, avoid making further changes. Any adjustments after testing would compromise the integrity of the test set, which is meant to be the unbiased evaluation of how well your model will perform in real-world scenarios.

In short, once you test, you're done refining. This is the point at which you can confidently assess the model’s true predictive power.

In [4]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8979 - loss: 0.2518 


In [5]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.26. Test accuracy: 89.73%
